In [4]:
ctx = SparkSession.builder \
    .master("spark://h:7077") \
    .appName("bosch spark") \
    .getOrCreate()
file_dir = "/home/h/kaggle_fun/bosch/raw_data/"

In [66]:
def load_csv_df(file_name):
    return ctx.read.csv(file_dir + file_name, header=True, inferSchema=True)

def check_p_n_cnt(input_df, col_name, pos):
    tmp_table_name = "check_p_n_input_df"
    input_df.registerTempTable(tmp_table_name)
    total = ctx.sql("select count(*) as tcnt from %s" % tmp_table_name)
    pos = ctx.sql("select count(*) as pcnt from %s where %s = %s" %(tmp_table_name, col_name, pos))
    print "positive/total : %s/%s" % (pos.collect()[0]["pcnt"], total.collect()[0]["tcnt"])

def adjust_p_n_ratio(input_df, col_name, pos):
    tmp_table_name = "adjust_p_n_input_df"
    input_df.registerTempTable(tmp_table_name)
    adjusted_df = ctx.sql("select * from %s" % tmp_table_name)
    print "positive/total : %s/%s" % (pos.collect()[0]["pcnt"], total.collect()[0]["tcnt"])
    

In [53]:
train_numer_df = load_csv_df("train_numeric.csv")
test_numer_df = load_csv_df("test_numeric.csv")

In [67]:
check_p_n_cnt(train_numer_df, "Response", "1")

positive/total : 6879/1183747


In [14]:
def get_corr_matrix(df):
    return df.stat.corr(df, method="pearson")

In [47]:
from pyspark.mllib.stat import Statistics
import numpy as np
from pyspark.mllib.linalg import Vectors
train_num_vec = train_numer_df.rdd.map(lambda data: Vectors.dense([float(c) for c in data]))
summary = Statistics.colStats(train_num_vec)

#Statistics.corr(train_numer_df)

Py4JJavaError: An error occurred while calling o388.colStats.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 32.0 failed 1 times, most recent failure: Lost task 2.0 in stage 32.0 (TID 573, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/h/soft/spark/python/lib/pyspark.zip/pyspark/worker.py", line 172, in main
    process()
  File "/home/h/soft/spark/python/lib/pyspark.zip/pyspark/worker.py", line 167, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/h/soft/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-47-e5100aaa45eb>", line 4, in <lambda>
TypeError: float() argument must be a string or a number

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:193)
	at org.apache.spark.api.python.PythonRunner$$anon$1.<init>(PythonRDD.scala

In [48]:
train_numer_df.head

<bound method DataFrame.head of DataFrame[Id: int, L0_S0_F0: double, L0_S0_F2: double, L0_S0_F4: double, L0_S0_F6: double, L0_S0_F8: double, L0_S0_F10: double, L0_S0_F12: double, L0_S0_F14: double, L0_S0_F16: double, L0_S0_F18: double, L0_S0_F20: double, L0_S0_F22: double, L0_S1_F24: double, L0_S1_F28: double, L0_S2_F32: double, L0_S2_F36: double, L0_S2_F40: double, L0_S2_F44: double, L0_S2_F48: double, L0_S2_F52: int, L0_S2_F56: double, L0_S2_F60: double, L0_S2_F64: double, L0_S3_F68: double, L0_S3_F72: double, L0_S3_F76: double, L0_S3_F80: double, L0_S3_F84: double, L0_S3_F88: int, L0_S3_F92: double, L0_S3_F96: double, L0_S3_F100: double, L0_S4_F104: double, L0_S4_F109: double, L0_S5_F114: double, L0_S5_F116: double, L0_S6_F118: double, L0_S6_F122: double, L0_S6_F132: double, L0_S7_F136: double, L0_S7_F138: double, L0_S7_F142: double, L0_S8_F144: double, L0_S8_F146: double, L0_S8_F149: double, L0_S9_F155: double, L0_S9_F160: double, L0_S9_F165: double, L0_S9_F170: double, L0_S9_F175: